In [1]:
! pip install pycuda

     |████████████████████████████████| 1.6MB 9.8MB/s 
     |████████████████████████████████| 71kB 11.0MB/s 
     |████████████████████████████████| 481kB 46.1MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621766 sha256=1b6754dc2d75433533fc9ea50187adb683d6cb55deb6e5bb4abca43552911e17
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2021.1-py2.py3-none-any.whl size=60415 sha256=168575ebb309150582076966e0a0a7a31824f8d24e59ef020f62e64453b4eac8
  Stored in directory: /root/.cache/pip/wheels/5d/85/10/7da05d77b47f2cb503e5c4185ba82187d0b6b6736b3c6641e0
  Created wheel for mako: filename=Mako-1.1.4-py2.py3-none-any.whl size=75675 sha256=3537ee836a700536e6eccd19b17ba5108d2343504499f9af3b3d44278c692ecf
  Stored in directory: /root/.cache/pip/wheels/ad/10/d3/aeb26e20d19045e2a68e5d3cbb57432e11b5d9c92c99f98d47
Successfully built pycuda pytools mako


In [2]:
!pip install py-cpuinfo

     |████████████████████████████████| 102kB 6.1MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-7.0.0-cp36-none-any.whl size=20072 sha256=5f91860efa9d2f5b9ddc2ce7ca432d4e82969de2522c129921aa766f88fc3d36
  Stored in directory: /root/.cache/pip/wheels/f1/93/7b/127daf0c3a5a49feb2fecd468d508067c733fba5192f726ad1
Successfully built py-cpuinfo


In [3]:
import cpuinfo
cpuinfo.get_cpu_info()['brand_raw']

'Intel(R) Xeon(R) CPU @ 2.20GHz'

In [4]:
!nvidia-smi

Wed Jan 27 12:56:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [111]:
import pycuda.gpuarray as gpuarray
import pycuda.autoinit
from  pycuda import  driver
from pycuda.compiler import SourceModule
import numpy as np
import time
from pycuda.curandom import rand as curand

In [131]:
#алгоритм для GPU
mod = SourceModule("""
  __global__ void pi_gpu(double *x, double *y, double *res_gpu, const int n) {
        
        int count_gpu = 0;
        int idx = threadIdx.x + (blockIdx.x*blockDim.x);
        int thread_count = gridDim.x*blockDim.x;

        for (int i=idx; i<n; i += thread_count) {
          int v;
          v = pow(x[i], 2) + pow(y[i], 2);
          if (v < 1)
            count_gpu += 1;
          
        }

        atomicAdd(res_gpu, count_gpu);
  }    
""")

In [97]:
#алгоритм для CPU
def pi_cpu(x, y, n):
  count_cpu = 0
  v = x ** 2 + y ** 2
  for i in range(n):
    if v[i] < 1:
      count_cpu += 1
  res = count_cpu * 4/n 
  return res

In [164]:
def generate_points():
  print('Введите число точек: ')
  n = int(input())

  x_gpu = curand((n,), dtype=np.double) 
  y_gpu = curand((n,), dtype=np.double)
  x = x_gpu.get().astype(np.double)
  y = y_gpu.get().astype(np.double)
  return x, y, n

In [169]:
x, y, n = generate_points()

Введите число точек: 
1000000


In [170]:
cpu_start = time.time()
res_cpu = pi_cpu(x,y,n)
cpu_time = time.time() - cpu_start
print('Число Пи: ', res_cpu)
print('Время на CPU: ', round(cpu_time,5))

Число Пи:  3.142152
Время на CPU:  0.35473


In [171]:
block = (32, 1, 1)
grid = (int(n/(32 * block[0])), 1)

res_gpu = gpuarray.zeros((1,), dtype=np.double)
res_gpu  = res_gpu.get()

gpu = mod.get_function("pi_gpu")

gpu_start = time.time()
gpu(driver.In(x), driver.In(y), driver.Out(res_gpu), np.int32(n), block = block, grid = grid)
driver.Context.synchronize()
gpu_time = time.time() - gpu_start

res_gpu =  res_gpu[0] * 4/n
print('Число Пи: ', res_gpu)
print('Время на GPU: ', round(gpu_time,5))

Число Пи:  3.142152
Время на GPU:  0.01023


In [172]:
print('Ускорение: ', cpu_time/gpu_time)

Ускорение:  34.691801902630104
